# Madrid Housing Prices Analysis

Data on flats from https://www.kaggle.com/datasets/mirbektoktogaraev/madrid-real-estate-market/data

Airbnb data: http://insideairbnb.com/get-the-data/

Location data from API (for neighborhoods and streets): https://here.com (geocoding)

## Initialization

In [ ]:
import pandas as pd
import geopandas as gpd
import json
import folium
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load data
df_houses = pd.read_csv('houses_Madrid.csv')

In [ ]:
# Load locations that we got from an API
neighborhood_locations = {}

with open("saved_locations.json", 'r') as f:
    neighborhood_locations = json.load(f)
    
street_locations = {}
with open("street_locations.json", 'r') as f:
    street_locations = json.load(f)

## Data analysis and visualization

In [ ]:
# Price VS number of rooms
nroom_group = df_houses.groupby('n_rooms')

nroom_mean_price = nroom_group['buy_price'].mean()
nroom_count = nroom_group.size()

fig, ax = plt.subplots(figsize=(15, 7))

ax.set_xlabel("Number of rooms")
ax.set_ylabel("Buy price, €")
ax.set_title("Price vs Number of rooms")

ax.bar(x=nroom_mean_price.index, height=nroom_mean_price)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))

ax.set_xlabel("Number of rooms")
ax.set_ylabel("Number of appartments")
ax.set_title("Number of appartments grouped by number of rooms in each")

ax.bar(x=nroom_count.index, height=nroom_count)

plt.show()

In [ ]:
# Price VS Area
group_areas = df_houses.groupby('sq_mt_built')

fig, ax = plt.subplots(figsize=(15, 7))

area_mean_price = group_areas['buy_price'].mean()

ax.set_title("Price vs Area")
ax.set_xlabel("Area (sq mt)")
ax.set_ylabel("Price, €")

ax.plot(area_mean_price.index, area_mean_price)

plt.show()

In [ ]:
# Number of rows VS neighborhood ids
neighborhood_group = df_houses.groupby('subtitle')

nhg_sizes = neighborhood_group.size()
nhg_sizes = nhg_sizes.sort_values(ascending=False)
plot, ax = plt.subplots(figsize=(22, 10))

plt.setp(ax.get_xticklabels(), rotation=60, ha='right')

ax.set_title("Number of houses in each neighborhood")
ax.bar(x=nhg_sizes.index, height=nhg_sizes)

plt.show()

### Visualizing street data

In [ ]:
# Initialize map
m = folium.Map(location=[40.37654, -3.60837], zoom_start=9.5)

In [ ]:
import colorsys

In [ ]:
def plot_dot(map: folium.Map, lat: float, lng: float, **kwargs):
    """Plot dot on map with parameters
    """
    rad = kwargs.get('radius')
    weight = kwargs.get('weight')
    color = kwargs.get('color')
    popup_text = kwargs.get('popup')
    
    folium.CircleMarker(location=[lat, lng],
                        radius=rad if rad is not None else 1,
                        weight=weight if weight is not None else 3,
                        color=color if color is not None else "#3388FF").add_to(map)\
                            .add_child(folium.Popup(popup_text))

def rgb_to_hexstr(r: int, g: int, b: int) -> str:
    """Converts RGB values to hex string
    """
    r = (r if r >= 0 else 0) if r <= 255 else 255
    g = (g if g >= 0 else 0) if g <= 255 else 255
    b = (b if b >= 0 else 0) if b <= 255 else 255
    return f'#{r:02x}{g:02x}{b:02x}'

def mapf(n: float, min1: float, max1: float, min2: float, max2: float) -> float:
    """Maps a value between (min1, max1) to (min2, max2)
    """
    return ((n - min1) / (max1 - min1)) * (max2 - min2) + min2

In [ ]:
# Clean houses DF
df_houses_clean = df_houses.loc[df_houses['buy_price'].notna() & \
    df_houses['sq_mt_built'].notna() & \
    df_houses['buy_price_by_area'].notna() & \
    (df_houses['sq_mt_built'] > 60.0) & \
    (df_houses['sq_mt_built'] < 100.0)]
# we cut everything lower than 60sqmt and above 100sqmt

In [ ]:
# Extract neighborhood name from 'subtitle' column

df_houses_clean['neighbourhood'] = df_houses_clean['subtitle'].str.extract(r"(.*), Madrid")[0]

In [ ]:
# Find min/max prices
min_price = df_houses_clean['buy_price'].min()
max_price = df_houses_clean['buy_price'].max()

min_price_by_area = df_houses_clean['buy_price_by_area'].min()
max_price_by_area = df_houses_clean['buy_price_by_area'].max()

In [ ]:
gdf_districts = gpd.GeoDataFrame.from_file("neighbourhoods.geojson")

In [ ]:
df_house_group = df_houses_clean.groupby('neighbourhood')
df_house_prices = df_house_group.mean(numeric_only=True).reset_index()

In [ ]:
df_filtered = df_house_prices.loc[
    df_house_prices['neighbourhood'].isin(gdf_districts['neighbourhood'])
]

In [ ]:
# District Choropleth

ENABLE_DISTRICTS = True

if ENABLE_DISTRICTS:
    # NOTE: Data can be wrong because district calculation is messy
    # Nothing I can do about that because dataset doesn't provide good enough data
    # to join with my GDF
    choropleth = folium.Choropleth(
        geo_data = gdf_districts,
        data = df_filtered,
        columns = ['neighbourhood', 'buy_price'],
        key_on = 'feature.properties.neighbourhood',
        fill_opacity = 0.7,
        fill_color = 'Greens',
        name = 'Prices',
        line_opacity = 1.0,
        line_weight = 2.0,
        legend_name = "Neighborhood average prices",
        smooth_factor = 0.0
    ).add_to(m)

In [ ]:
# Process values and add them to the map
for street in street_locations.values():
    try:
        data = df_houses_clean.iloc[street['idx']]
    except IndexError:
        continue # skip if index isn't in dataframe
    mapped_price = mapf(data['buy_price_by_area'], min_price_by_area, max_price_by_area, 0.0, 1.0) ** 0.5
    color = [int(c * 255) for c in colorsys.hsv_to_rgb(0.3, 0.5 + mapped_price * 0.5, 1.0 - mapped_price)]
    plot_dot(m, street['loc']['lat'], street['loc']['lng'], 
             radius=2, weight=4, color=rgb_to_hexstr(color[0], color[1], color[2]),
             popup=f'Price per SqMt: {data['buy_price_by_area']}')

In [ ]:
m

This dataset has a lot of missing and wrong data. It is useful in some way, but
it isn't as useful as I'd initially thought.

## Second dataset

Next is analysis of rent prices of ABNB listings for 2023

In [ ]:
df_abnb_listings = pd.read_csv("AirbnbListingsNew.csv")

In [ ]:
gdf_neighborhoods: gpd.GeoDataFrame = gpd.GeoDataFrame.from_file('neighbourhoods.geojson')

In [ ]:
df_abnb_price_clean = df_abnb_listings.loc[(df_abnb_listings['price'].notna()) & (df_abnb_listings['price'] < 500.0) & (df_abnb_listings['price'] > 15.0)]

In [ ]:
import re

def extract_bedrooms_from_title(title: str) -> int | None:
    """Extracts bedroom number from title (for calculations)
    """
    rgx = re.compile(r"(\d+)\sbedroom")
    rgx_studio = re.compile(r"[Ss]tudio")
    
    rgx_match = rgx.search(title)
    
    if rgx_match is None:
        if rgx_studio.search(title) is None:
            return None
        return 1
    
    rgx_num = rgx_match.group(1)

    return int(rgx_num)

In [ ]:
# Extract bedrooms

bedroom_counts = []
prices_per_bedroom = []
for idx, listing in df_abnb_price_clean.iterrows():
    bedroom_count = extract_bedrooms_from_title(listing['name'])
    if bedroom_count is None:
        bedroom_counts.append(np.nan)
        prices_per_bedroom.append(np.nan)
        continue
    bedroom_counts.append(float(bedroom_count))
    prices_per_bedroom.append(listing['price'] / bedroom_count)

In [ ]:
# Clean prices more
df_abnb_price_clean = df_abnb_price_clean.assign(bedroom_count=bedroom_counts,
                                                 price_per_bedroom=prices_per_bedroom)
df_abnb_price_clean = df_abnb_price_clean.loc[
    df_abnb_price_clean['bedroom_count'].notna() & \
    df_abnb_price_clean['price_per_bedroom'].notna()
]

df_abnb_price_clean['bedroom_count_str'] = df_abnb_price_clean['bedroom_count'].astype(str)
df_abnb_price_clean.loc[df_abnb_price_clean['bedroom_count'] >= 4, 'bedroom_count_str'] = '>4'

In [ ]:
# NOTE: Used to be constrained for calculations. Now deprecated since we group data
df_abnb_room_constraint = df_abnb_price_clean

In [ ]:
# Calculate prices

min_price = df_abnb_room_constraint['price'].min()
max_price = df_abnb_room_constraint['price'].max()

min_price_per_bedroom = df_abnb_room_constraint['price_per_bedroom'].min()
max_price_per_bedroom = df_abnb_room_constraint['price_per_bedroom'].max()

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
# Calculate and visualize points on a map
new_map = folium.Map(location=[40.37654, -3.60837], zoom_start=9.5)

In [ ]:
# Make lat/lng points into list
points = [Point(listing['longitude'], listing['latitude']) for _, listing in df_abnb_room_constraint.iterrows()]

In [ ]:
gdf_points = gpd.GeoDataFrame(df_abnb_room_constraint, geometry=points, crs=4326)

# Get points WITHIN neighborhoods
gdf_points_in = gpd.sjoin(gdf_points, gdf_neighborhoods, predicate='within')

In [ ]:
# Group neighborhoods and get average price per bedroom
gdf_neigborhood_group = gdf_points_in.groupby('neighbourhood_right')

df_neighborhood_mean_price = gdf_neigborhood_group['price_per_bedroom'].mean()
df_neighborhood_flats_count = gdf_neigborhood_group.size()

In [ ]:
# Count listings in neighborhood with specific amount of rooms

gdf_neigborhood_groups_group = gdf_points_in.groupby('neighbourhood_group_right')

df_bedroom_count_groups = pd.DataFrame(gdf_neigborhood_groups_group[['bedroom_count_str']].value_counts())

df_bedroom_count_groups['percentage'] = 100.0 * df_bedroom_count_groups['count'] / gdf_neigborhood_groups_group['bedroom_count'].size()

# Plot percentage of distribution of bedrooms
df_bedroom_pivot_table = df_bedroom_count_groups.pivot_table(values='percentage', index='neighbourhood_group_right', columns=['bedroom_count_str'])
df_bedroom_pivot_table.plot(kind='bar', stacked=True, title='Percentage of flats with specific number of rooms in each neighbourhood')

plt.show()

In [ ]:
df_bedroom_pivot_table = df_bedroom_count_groups.pivot_table(values='count', index='neighbourhood_group_right', columns=['bedroom_count_str'])
df_bedroom_pivot_table = df_bedroom_pivot_table.sort_values('1.0', ascending=False)
df_bedroom_pivot_table.plot(kind='bar', stacked=True, title='Number of flats with specific number of rooms in each neighbourhood')

plt.show()

In [ ]:
# Prepare grouped dataframes

df_bedrooms_single = gdf_points_in.loc[gdf_points_in['bedroom_count_str'] == '1.0']
df_bedrooms_two = gdf_points_in.loc[gdf_points_in['bedroom_count_str'] == '2.0']
df_bedrooms_three = gdf_points_in.loc[gdf_points_in['bedroom_count_str'] == '3.0']
df_bedrooms_more = gdf_points_in.loc[gdf_points_in['bedroom_count_str'] == '>4']

single_prices = df_bedrooms_single.groupby('neighbourhood_group_right')['price_per_bedroom'].mean()
two_prices = df_bedrooms_two.groupby('neighbourhood_group_right')['price_per_bedroom'].mean()
three_prices = df_bedrooms_three.groupby('neighbourhood_group_right')['price_per_bedroom'].mean()
more_prices = df_bedrooms_more.groupby('neighbourhood_group_right')['price_per_bedroom'].mean()

single_prices.name = '1'
two_prices.name = '2'
three_prices.name = '3'
more_prices.name = '>4'

mean_price_all = gdf_neigborhood_groups_group['price_per_bedroom'].mean()
mean_price_all.name = 'all'

df_prices_data = pd.concat([single_prices, two_prices, three_prices, more_prices, mean_price_all], axis=1).reset_index()

In [ ]:
# Display distribution of number of bedrooms for each neighborhood

plt.figure(figsize=(15, 10))
ax = sns.scatterplot(df_prices_data.sort_values('1', ascending=False)
                     .set_index('neighbourhood_group_right'))
ax.set_title('Average price of each group of flats in each neighborhood')
ax.set_ylabel('Price per bedroom, €')
ax.set_xlabel("District name")

ax.set_xticks(df_prices_data['neighbourhood_group_right'])
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Show mean price on map

folium.Choropleth(
    geo_data=gdf_neighborhoods,
    name='choropleth',
    data=df_neighborhood_mean_price,
    columns=['neighbourhood_group_right', 'price_per_bedroom'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=1.0,
    line_weight=2.0,
    legend_name="Neighborhood mean price per bedroom"
).add_to(new_map)

In [ ]:
# Plot dots for each listing

PLOT_DOTS = True

if PLOT_DOTS:
    for name, listing in df_abnb_room_constraint.iterrows():
        price_per_bedroom = listing['price_per_bedroom']
        mapped_price = mapf(price_per_bedroom, min_price_per_bedroom, max_price_per_bedroom, 0.0, 1.0) ** 0.5
        color = [int(c * 255) for c in colorsys.hsv_to_rgb(0.3, 0.5 + mapped_price * 0.5, 1.0 - mapped_price)]
        plot_dot(new_map, listing['latitude'], listing['longitude'], 
                color=rgb_to_hexstr(color[0], color[1], color[2]), 
                radius=1, weight=2, popup=f'Price per bedroom: {listing['price_per_bedroom']}\nTotal price: {listing['price']}')

In [ ]:
new_map

## Third dataset

Analysis of real-estate transactions per district

In [ ]:
df_transactions = pd.read_csv("data.csv")

In [ ]:
# Constrain transactions to most recent
df_transactions = df_transactions.loc[pd.to_datetime(df_transactions['datetime'], format="%m/%Y") > '2022-01-01']

In [ ]:
# df_transactions = df_transactions.head(30000) # limit for debugging

In [ ]:
points = [Point(transaction['longitude'], transaction['latitude']) for _, transaction in df_transactions.iterrows()]

In [ ]:
gdf_points = gpd.GeoDataFrame(df_transactions, geometry=points, crs=4326)

# Get points WITHIN neighborhoods
gdf_points_in = gpd.sjoin(gdf_points, gdf_neighborhoods, predicate='within')

# Calculate area because it isn't in the dataset
gdf_points_in['area_sqm'] = (gdf_points_in['value'] / gdf_points_in['value_sqm'])

In [ ]:
# Constrain data for visualization

# Constrained by: 70 < area < 100
gdf_points_constrained = gdf_points_in.loc[(gdf_points_in['area_sqm'] < 100.0) & \
    (gdf_points_in['area_sqm'] > 70.0)]

In [ ]:
# Get groups for calculations and visualizations and find mean

dg_neighborhood = gdf_points_constrained.groupby('neighbourhood')
dg_neighborhood_group = gdf_points_constrained.groupby('neighbourhood_group')

mean_per_neighborhood = dg_neighborhood.mean(numeric_only=True)
mean_per_group = dg_neighborhood_group.mean(numeric_only=True)

In [ ]:
plt.figure(figsize=(10, 20))
ax = sns.violinplot(data=gdf_points_constrained.sort_values('area_sqm'),
               x='area_sqm', y='neighbourhood_group',
               cut=0)

ax.set_title("Distribution of area per flat in each district")

plt.show()
plt.close()

In [ ]:
# Clean values

df_mean_per_neighborhood = mean_per_neighborhood.reset_index()
df_mean_per_group = mean_per_group.reset_index()

df_mean_per_group = df_mean_per_group.sort_values('value', ascending=False)

df_flat_count_neighborhood = dg_neighborhood.size()
df_flat_count_neighborhood_group = dg_neighborhood_group.size()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['figure.dpi'] = 100

ax = plt.subplot()

ax.set_xticks(np.arange(df_mean_per_group['neighbourhood_group'].index.size))
ax.set_xticklabels(df_mean_per_group['neighbourhood_group'], rotation=45)
ax.bar(x=df_mean_per_group['neighbourhood_group'],
       height=df_mean_per_group['value'], label='Euro')

ax.set_title('Mean value each transaction per neighborhood')
ax.set_ylabel('Mean Price, Euro')
ax.set_xlabel('Neighborhood group')

ax.legend()

plt.show()
plt.close()

In [ ]:
gdf_value_sqm_constrained = gdf_points_constrained.loc[
    gdf_points_constrained['value_sqm'] < 10000]

In [ ]:
ax = sns.scatterplot(data=gdf_value_sqm_constrained,
                x='construction_year', y='value_sqm', hue='value', alpha=0.4)
ax.set_title("Value of sqm vs Construction year")
ax.grid()
plt.show()

In [ ]:
ax = sns.histplot(data=gdf_value_sqm_constrained,
             x='construction_year', y='value_sqm')

ax.set_title("Value of sqm vs Construction year")
plt.show()

In [ ]:
# Price by square meter cut off prices more than 10k per sqm

ax = sns.boxplot(data=gdf_points_constrained.loc[gdf_points_constrained['value_sqm'] < 10000], 
            order=df_mean_per_group['neighbourhood_group'],
            y='neighbourhood_group', x='value_sqm')
ax.set_title("Price per sqm in each neighborhood")
plt.show()

In [ ]:
gdf_constrained_points_price = gdf_points_constrained.loc[
    gdf_points_constrained['value'] < 1_000_000
]

In [ ]:
# Mean value of transactions in neighborhood for >2022-01-01

ax = sns.boxplot(data=gdf_constrained_points_price, 
            order=df_mean_per_group['neighbourhood_group'],
            y='neighbourhood_group', x='value')

ax.set_title("Price in each neighborhood")
plt.show()

In [ ]:
map_fol = folium.Map(location=[40.37654, -3.60837], zoom_start=9.5)

folium.Choropleth(
    geo_data=gdf_neighborhoods,
    name='choropleth',
    data=df_mean_per_neighborhood,
    columns=['neighbourhood', 'value_sqm'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=1.0,
    line_weight=2.0,
    legend_name="Price per sqm"
).add_to(map_fol)

In [ ]:
map_fol

## Conclusion

Apparently, Salamanca is the most expensive area, as indicated by most datasets.
This is apparently the case because Salamanca has the biggest share of bigger
flats, as shown in "Percentage of flats with specific number of rooms in each neighbourhood"
figure.

Generally, as shown on the map, southern areas are cheaper than areas in north and center.
As shown in area distribution plots, southern areas have smaller flats than
central/northern areas. This is probably the reason why these areas are cheaper.

Central areas, including Center, Salamanca, Retiro etc., are generally more expensive
than other areas.

Scatter plot of "Value sqm vs Construction year" shows an interesting insight on
history of real estate market in Madrid. There is a clear line on year 1950,
before which there aren't as many flats as after. It can be safe to assume that
everything before that line could be historical buildings.

Rental estate, as indicated by ABnB dataset, shows a similar picture with small
differences. E.g. flats in Latina are more expensive to rent, than to buy, relative to other
districts.